In [1]:
!pip install duckdb --no-index --find-links=file:///kaggle/input/birdclef24-duckdb-polars/kaggle/working/mysitepackages/duck_pkg
!pip install polars --no-index --find-links=/kaggle/input/birdclef24-duckdb-polars/kaggle/working/mysitepackages/polars_pkg

Looking in links: file:///kaggle/input/birdclef24-duckdb-polars/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/birdclef24-duckdb-polars/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: /kaggle/input/birdclef24-duckdb-polars/kaggle/working/mysitepackages/polars_pkg


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import duckdb as dd
import polars as pl
import pyarrow
import os
import glob
import shutil
import zipfile
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
import plotly.express as px
import librosa
from IPython.display import Audio
import pickle
from joblib import dump, load
from pathlib import Path
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [3]:
# Path to the directory containing your audio dataset
dataset_dir = '/kaggle/input/birdclef-2024/train_audio'
# Initialize an empty dictionary to store the mapping between audio files and labels
label_mapping = {}
# Iterate over subdirectories (classes) in the dataset directory
for label in os.listdir(dataset_dir):
    label_dir = os.path.join(dataset_dir, label)
    # Check if the item in the dataset directory is a directory
    if os.path.isdir(label_dir):
        # Iterate over audio files in the subdirectory (class)
        for audio_file in os.listdir(label_dir):
            # Add the mapping between audio file path and label to the dictionary
            audio_file_path = os.path.join(label_dir, audio_file)
            label_mapping[audio_file_path] = label
            
# label_mapping

# Create a list of tuples containing the audio file paths and labels
data = [(audio_file_path, label) for audio_file_path, label in label_mapping.items()]
# Create a Pandas DataFrame from the list of tuples
annotated_data = pd.DataFrame(data, columns=['audio_file_path', 'label'])

label_encoder = LabelEncoder()
annotated_data['encoded_label'] = label_encoder.fit_transform(annotated_data['label'])

print(annotated_data.head(5))
print(annotated_data.shape)

                                     audio_file_path    label  encoded_label
0  /kaggle/input/birdclef-2024/train_audio/redspu...  redspu1            137
1  /kaggle/input/birdclef-2024/train_audio/redspu...  redspu1            137
2  /kaggle/input/birdclef-2024/train_audio/redspu...  redspu1            137
3  /kaggle/input/birdclef-2024/train_audio/redspu...  redspu1            137
4  /kaggle/input/birdclef-2024/train_audio/redspu...  redspu1            137
(24459, 3)


In [4]:
# Path to the directory containing your audio dataset
dataset_dir = '/kaggle/input/birdclef-2024/unlabeled_soundscapes'
# Initialize an empty dictionary to store the mapping between audio files and labels
label_mapping = {}

for audio_file in os.listdir(dataset_dir):
    # Add the mapping between audio file path and label to the dictionary
    audio_file_path = os.path.join(dataset_dir, audio_file)
    label_mapping[audio_file_path] = 'unlabelled'
            
# label_mapping

# Create a list of tuples containing the audio file paths and labels
data = [(audio_file_path, label) for audio_file_path, label in label_mapping.items()]
# Create a Pandas DataFrame from the list of tuples
unannotated_data = pd.DataFrame(data, columns=['audio_file_path', 'label'])

# label_encoder = LabelEncoder()
unannotated_data['encoded_label'] = 999

print(unannotated_data.head(5))
print(unannotated_data.shape)

                                     audio_file_path       label  \
0  /kaggle/input/birdclef-2024/unlabeled_soundsca...  unlabelled   
1  /kaggle/input/birdclef-2024/unlabeled_soundsca...  unlabelled   
2  /kaggle/input/birdclef-2024/unlabeled_soundsca...  unlabelled   
3  /kaggle/input/birdclef-2024/unlabeled_soundsca...  unlabelled   
4  /kaggle/input/birdclef-2024/unlabeled_soundsca...  unlabelled   

   encoded_label  
0            999  
1            999  
2            999  
3            999  
4            999  
(8444, 3)


In [6]:
dd.sql("select label, count(distinct(audio_file_path)) as files from annotated_data group by label order by 2").pl()

label,files
str,i64
"""niwpig1""",5
"""asiope1""",5
"""integr""",5
"""blaeag1""",6
"""wynlau1""",6
…,…
"""grywag""",500
"""graher1""",500
"""grnsan""",500


In [ ]:
li = []

path = '/kaggle/input/kmeans-labelled-files'
# all_files = glob.glob(os.path.join(path , "/*.csv"))

for label in os.listdir(path):
    label_dir = os.path.join(path, label)
    #print(label_dir)
    df = pd.read_csv(label_dir, index_col=None, header=0)
    li.append(df)
    
frame = pd.concat(li, axis=0, ignore_index=True)
print(frame.shape)

In [ ]:
len(dd.sql("select distinct label from frame").pl().to_series().to_list())

In [ ]:
import warnings
warnings.filterwarnings("ignore")

list_of_birds = dd.sql("select distinct label from annotated_data where label not in (select distinct label from frame) \
limit 30").pl().to_series().to_list()

def get_file_duration(full_file_path):
    duration = librosa.get_duration(path=full_file_path, sr=32000)
    return round(duration,2)

def set_file_duration_in_df(input_df):
    vect_func = np.vectorize(get_file_duration)
    input_df_pd = input_df.to_pandas()
    input_df_pd['file_duration'] = vect_func(input_df_pd['audio_file_path'])
    return input_df_pd

def get_features(file_df):
    features = []
    for i in range(file_df.shape[0]):
        file_path = file_df.item(i,0)
        audio, sample_rate = librosa.load(file_path, sr=32000)
        samples_per_segment = sample_rate * 5
        if len(audio) > 7680000:
            total_samples = 7680000
        else:
            total_samples = len(audio)

        for j in range(0, total_samples+160000, samples_per_segment):
            if j + samples_per_segment <= total_samples:
                segment = audio[j:j + samples_per_segment]
                mfccs = librosa.feature.mfcc(y=segment, sr=32000, n_mfcc=40)
                flattened_features = (np.mean(mfccs.T, axis=0))
                features.append({'file_path': file_path, 'label': file_df.item(i,2), 'feature_vector': flattened_features})
    
    features_df = pd.DataFrame.from_dict(features)
    return features_df

def run_kmeans(feature_matrix, features_df):
    n_clusters = [10, 27, 30, 35, 42, 47, 50, 55, 75, 95, 100, 105, 110, 125, 135]
    metrics_tracking = {}
    metrics_tracking['n_clusters'] = n_clusters
    list_inertia = []
    list_sh_score = []

    for cluster in n_clusters:
        kmeans = KMeans(n_clusters = cluster, init='k-means++', random_state=42)
        kmeans.fit(feature_matrix)
        list_inertia.append(kmeans.inertia_)
        sh_score = silhouette_score(feature_matrix, kmeans.labels_)
        list_sh_score.append(sh_score)

    metrics_tracking['inertia'] = list_inertia
    metrics_tracking['silhouette_score'] = list_sh_score

    test_df = pd.DataFrame(metrics_tracking)
    optimum_n_clusters = test_df[test_df['silhouette_score']==test_df['silhouette_score'].max()]['n_clusters'].values[0]
    
    kmeans = KMeans(n_clusters = optimum_n_clusters, init='k-means++', random_state=42)
    kmeans.fit(feature_matrix)
    
    features_df['cluster'] = kmeans.labels_
    
    def get_cluster_centroids(cluster_id):
        return kmeans.cluster_centers_[cluster_id]
    
    features_df['centroid'] = features_df['cluster'].apply(get_cluster_centroids)
    
    def get_ftr_w_centroid(feature, cluster_id):
        list_of_ftr_n_cc = [feature, kmeans.cluster_centers_[cluster_id]]
        return list_of_ftr_n_cc
    
    features_df['ftr_w_centroid'] = features_df.apply(lambda x: get_ftr_w_centroid(x['feature_vector'], x['cluster']), axis=1)

    def calc_cos_sim(ftr_w_centroid):
        v = ftr_w_centroid[0]
        w = ftr_w_centroid[1]
        cos_before_norm = np.dot(v, w) / (np.linalg.norm(v) * np.linalg.norm(w))
        rounded_cos_sim = round(cos_before_norm, 2)
        return rounded_cos_sim
    
    features_df['cosine_similarity'] = features_df['ftr_w_centroid'].apply(calc_cos_sim)
    
    return features_df

In [ ]:
## Pipeline

for bird in list_of_birds:
    print(bird)
    combined_data_sql = "select * from annotated_data where label = '{0}' union select * from unannotated_data".format(bird)
    combined_data = dd.sql(combined_data_sql).pl()
    combined_data_with_duration = set_file_duration_in_df(combined_data)
    
    files_per_species_w_rnk = dd.sql("select *, row_number()over(partition by label, encoded_label order by file_duration desc) as rn \
    from combined_data_with_duration where file_duration <= 240").pl().sort(by=['encoded_label','rn'])

    files_per_species_final = files_per_species_w_rnk.filter(pl.col('rn')<=35)
    
    features_df = get_features(files_per_species_final)
    
    feature_matrix = np.vstack(features_df.feature_vector.values)
    
    features_df_with_clusters = run_kmeans(feature_matrix, features_df)
    
    list_of_clusters = dd.sql(" select distinct cluster from ( select *, row_number()over(order by files desc) as rn from \
    ( select cluster, count(distinct(label)) as data_count, count(distinct(file_path)) as files \
    from features_df_with_clusters group by cluster )t1 )t2 where rn <= 5").pl().to_series().to_list()
    
    bird_label = dd.sql("select distinct label from features_df_with_clusters where label != 999").pl().item()
    
    features_df_with_clusters['label'] = np.where(features_df_with_clusters['cluster'].isin(list_of_clusters)\
                                                  , bird_label, features_df_with_clusters['label'])
    
    final_list_of_files_sql = "select distinct file_path from features_df where label = {0}".format(bird_label)
    final_list_of_files = dd.sql(final_list_of_files_sql).pl().to_series().to_list()
    
    combined_data_with_duration['label'] = \
    np.where(combined_data_with_duration['audio_file_path'].isin(final_list_of_files), bird, combined_data_with_duration['label'])

    combined_data_with_duration['encoded_label'] = \
    np.where(combined_data_with_duration['audio_file_path'].isin(final_list_of_files), bird_label \
             , combined_data_with_duration['encoded_label'])
    
    combined_data_final = combined_data_with_duration[combined_data_with_duration['audio_file_path'].isin(final_list_of_files)]\
    .reset_index(drop=True)
    
    csv_name = "combined_data_final_{0}.csv".format(bird)
    combined_data_final.to_csv(csv_name, index=False)
    print("finished pipeline for bird : {0}".format(bird))

In [ ]:
combined_data = dd.sql("select * from annotated_data where label = 'blaeag1' union select * from unannotated_data").pl()
combined_data.shape

In [ ]:
def get_file_duration(full_file_path):
    duration = librosa.get_duration(path=full_file_path, sr=32000)
    return round(duration,2)

vect_func = np.vectorize(get_file_duration)

combined_data_pd = combined_data.to_pandas()

combined_data_pd['file_duration'] = vect_func(combined_data_pd['audio_file_path'])
print(combined_data_pd.shape)

In [ ]:
files_per_species_w_rnk = dd.sql(" select *, row_number()over(partition by label, encoded_label order by file_duration desc) as rn \
from combined_data_pd where file_duration <= 240").pl().sort(by=['encoded_label','rn'])

files_per_species_final = files_per_species_w_rnk.filter(pl.col('rn')<=35)

print(files_per_species_final.shape)

In [ ]:
from tqdm import tqdm

#labels = []
features = []
#feature_dict = {}

# Loop through each audio file in the dataset directory
for i in tqdm(range(files_per_species_final.shape[0]), desc = 'Outer Loop'):
    # labels.append(annotated_data['label'].iloc[i])
    file_path = files_per_species_final.item(i,0)
    # lat = files_per_species_final.item(i,3)
    # lon = files_per_species_final.item(i,4)
    audio, sample_rate = librosa.load(file_path, sr=32000)
    samples_per_segment = sample_rate * 5
    if len(audio) > 7680000:
        total_samples = 7680000
    else:
        total_samples = len(audio)

    for j in range(0, total_samples+160000, samples_per_segment):
        if j + samples_per_segment <= total_samples:
            segment = audio[j:j + samples_per_segment]
            # chunk = cur_file+'_'+str(int((j/32000)+5))
            # mfccs = librosa.feature.mfcc(y=segment, sr=32000, n_mfcc=40)
            # flattened_features = (np.mean(mfccs.T, axis=0))
            # melspec = librosa.feature.melspectrogram(y=segment, sr=32000, n_fft=500, hop_length = 50)
            mfccs = librosa.feature.mfcc(y=segment, sr=32000, n_mfcc=40)
            # melspec = librosa.feature.melspectrogram(y=segment, sr=sr, n_fft=n_fft, n_mels=n_mels, hop_length=hop_length, fmin=fmin, fmax=fmax)
            # mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_fft=n_fft, n_mels=n_mels, hop_length=hop_length, fmin=fmin, fmax=fmax)
            # flattened_melspec_features = (np.mean(melspec.T, axis=0))
            """flattened_mfcc_features = (np.mean(mfccs.T, axis=0))
            flattened_features = np.append(flattened_melspec_features, flattened_mfcc_features)"""
            # features.append(np.append(flattened_features, np.array([lat,lon])))
            flattened_features = (np.mean(mfccs.T, axis=0))
            #features.append(flattened_features)
            #labels.append(files_per_species_final.item(i,2))
            features.append({'file_path': file_path, 'label': files_per_species_final.item(i,2), 'feature_vector': flattened_features})
            

In [ ]:
features_df = pd.DataFrame.from_dict(features)
print(features_df.shape)
print(features_df.head(5))
print(features_df.tail(5))

In [ ]:
dd.sql("select distinct label from features_df where label != 999").pl().item()

In [ ]:
feature_matrix = np.vstack(features_df.feature_vector.values)

In [ ]:
from sklearn.metrics import silhouette_score

n_clusters = [10, 27, 30, 35, 42, 47, 50, 55, 75, 95, 100, 105, 110, 125, 135]
metrics_tracking = {}
metrics_tracking['n_clusters'] = n_clusters
list_inertia = []
list_sh_score = []

for cluster in n_clusters:
    print("starting kmeans with {0} clusters".format(cluster))
    kmeans = KMeans(n_clusters = cluster, init='k-means++', random_state=42)
    kmeans.fit(feature_matrix)
    
    print("inertia = {0}".format(kmeans.inertia_))
    list_inertia.append(kmeans.inertia_)
    
    sh_score = silhouette_score(feature_matrix, kmeans.labels_)
    print("silhouette score = {0}".format(sh_score))
    list_sh_score.append(sh_score)
    
metrics_tracking['inertia'] = list_inertia
metrics_tracking['silhouette_score'] = list_sh_score

test_df = pd.DataFrame(metrics_tracking)

In [ ]:
test_df[test_df['silhouette_score']==test_df['silhouette_score'].max()]['n_clusters'].values[0]

In [ ]:
fig = px.line(test_df, x='n_clusters', y='silhouette_score', markers=True)
fig.show()

In [ ]:
fig = px.line(test_df, x='n_clusters', y='inertia', markers=True)
fig.show()

In [ ]:
kmeans = KMeans(n_clusters = 50, init='k-means++', random_state=42)
kmeans.fit(feature_matrix)

In [ ]:
features_df['cluster'] = kmeans.labels_

def get_cluster_centroids(cluster_id):
    return kmeans.cluster_centers_[cluster_id]

features_df['centroid'] = features_df['cluster'].apply(get_cluster_centroids)

def get_ftr_w_centroid(feature, cluster_id):
    list_of_ftr_n_cc = [feature, kmeans.cluster_centers_[cluster_id]]
    return list_of_ftr_n_cc

features_df['ftr_w_centroid'] = features_df.apply(lambda x: get_ftr_w_centroid(x['feature_vector'], x['cluster']), axis=1)

def calc_cos_sim(ftr_w_centroid):
    v = ftr_w_centroid[0]
    w = ftr_w_centroid[1]
    
    cos_before_norm = np.dot(v, w) / (np.linalg.norm(v) * np.linalg.norm(w))
    rounded_cos_sim = round(cos_before_norm, 2)
    return rounded_cos_sim

features_df['cosine_similarity'] = features_df['ftr_w_centroid'].apply(calc_cos_sim)

In [ ]:
list_of_clusters = dd.sql(" select distinct cluster from ( select *, row_number()over(order by files desc) as rn from \
( select cluster, count(distinct(label)) as data_count, count(distinct(file_path)) as files \
from features_df group by cluster )t1 )t2 where rn <= 5").pl().to_series().to_list()

print(list_of_clusters)

In [ ]:
features_df['label'] = np.where(features_df['cluster'].isin(list_of_clusters), 16, features_df['label'])

In [ ]:
final_list_of_files = dd.sql("select distinct file_path from features_df where label = 16 ").pl().to_series().to_list()

print(final_list_of_files)

In [ ]:
features_df[features_df['file_path'].isin(final_list_of_files)]

In [ ]:
combined_data_pd['label'] = \
np.where(combined_data_pd['audio_file_path'].isin(final_list_of_files), 'blaeag1', combined_data_pd['label'])

combined_data_pd['encoded_label'] = \
np.where(combined_data_pd['audio_file_path'].isin(final_list_of_files), 16, combined_data_pd['encoded_label'])

combined_data_pd[combined_data_pd['audio_file_path'].isin(final_list_of_files)]

In [ ]:
combined_data_pd_blaeag1 = combined_data_pd[combined_data_pd['audio_file_path'].isin(final_list_of_files)].reset_index(drop=True)

In [ ]:
combined_data_pd_blaeag1.to_csv('combined_data_pd_blaeag1.csv', index=False)

In [7]:
li = []

path = '/kaggle/input/kmeans-labelled-files'
# all_files = glob.glob(os.path.join(path , "/*.csv"))

for label in os.listdir(path):
    label_dir = os.path.join(path, label)
    #print(label_dir)
    df = pd.read_csv(label_dir, index_col=None, header=0)
    li.append(df)
    
frame = pd.concat(li, axis=0, ignore_index=True)
print(frame.shape)

(7419, 4)


In [8]:
frame.head(10)

,audio_file_path,label,encoded_label,file_duration
0,/kaggle/input/birdclef-2024/unlabeled_soundsca...,rufbab3,145,240.0
1,/kaggle/input/birdclef-2024/unlabeled_soundsca...,rufbab3,145,240.0
2,/kaggle/input/birdclef-2024/unlabeled_soundsca...,rufbab3,145,240.0
3,/kaggle/input/birdclef-2024/unlabeled_soundsca...,rufbab3,145,240.0
4,/kaggle/input/birdclef-2024/unlabeled_soundsca...,rufbab3,145,240.0
5,/kaggle/input/birdclef-2024/unlabeled_soundsca...,rufbab3,145,240.0
6,/kaggle/input/birdclef-2024/unlabeled_soundsca...,rufbab3,145,240.0
7,/kaggle/input/birdclef-2024/unlabeled_soundsca...,rufbab3,145,240.0
8,/kaggle/input/birdclef-2024/unlabeled_soundsca...,rufbab3,145,240.0
9,/kaggle/input/birdclef-2024/unlabeled_soundsca...,rufbab3,145,240.0


In [9]:
## Test to see if the same file has been put under more than one class
pl.Config(fmt_str_lengths=100)
dd.sql(" select audio_file_path, count(distinct(encoded_label)) as encoded_label_count from frame \
where audio_file_path in (select audio_file_path \
from frame group by audio_file_path having count(distinct(encoded_label)) > 1) group by audio_file_path order by 2 desc").pl()

audio_file_path,encoded_label_count
str,i64
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1358642276.ogg""",59
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1574959233.ogg""",57
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1682474551.ogg""",54
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/869721194.ogg""",54
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/366413676.ogg""",53
…,…
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1132784106.ogg""",2
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/908548847.ogg""",2
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1400428116.ogg""",2


In [17]:
frame_with_rn = dd.sql("select *, row_number()over(partition by audio_file_path order by encoded_label) rn from frame").pl()

In [23]:
final_frame = dd.sql("select * from frame_with_rn where rn = 1").pl()

In [24]:
# these files need to stay for one label only
pl.Config(fmt_str_lengths=100)
dd.sql(" select distinct audio_file_path from final_frame where audio_file_path in (select audio_file_path \
from final_frame group by audio_file_path having count(distinct(encoded_label)) > 1) ").pl()

audio_file_path
str


In [25]:
pl.Config(fmt_str_lengths=100)
dd.sql(" select distinct audio_file_path from final_frame where audio_file_path like '%unlabeled%'").pl()

audio_file_path
str
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1015325399.ogg"""
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1083828063.ogg"""
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1123598635.ogg"""
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1233478899.ogg"""
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1291387146.ogg"""
…
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1822012588.ogg"""
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/440672859.ogg"""
"""/kaggle/input/birdclef-2024/unlabeled_soundscapes/1932411226.ogg"""
